In [66]:
# mocks the data for Pratham usecase V1 and recommends content based on content popularity
import csv
import sys
import collections
import os.path
import requests

# on exit clean-ups
import atexit

# cassandra libs
from cassandra.cluster import Cluster
from cassandra.query import dict_factory


# neo4j libs
from py2neo import Graph
from py2neo import Node, Relationship
from py2neo import authenticate
from random import randint
from numpy import np

# neo4j graph connector
authenticate("localhost:7474", "neo4j", "1sTep123")
graph = Graph()
# delete entire graph
#graph.delete_all()



# bool flag database connections
cassandraDbOn=False
neo4jDbOn=False
ASERdict_num={1:"ASERlevel_Beginner",2:"ASERlevel_Addition",3:"ASERlevel_Subtraction",4:"ASERlevel_Multiplication",5:"ASERlevel_Division"}
ASERdict_lit={1:"ASERlevel_Beginner",2:"ASERlevel_Letter",3:"ASERlevel_Word",4:"ASERlevel_Sentence",5:"ASERlevel_Paragraph"}
def dbCleanUP(cassandraDbOn,neo4jDbOn):
    if cassandraDbOn:
    	print 'cleaning Cassandra state'
    	session.shutdown();
    	cluster.shutdown();

atexit.register(dbCleanUP,True,True)

# setup cassandra connection
cassandraDbOn=True
cluster = Cluster()
session = cluster.connect('learner_db')

# set response schema to Dictionaries
session.row_factory = dict_factory

# process learner-db
# move content summary table
def movecontentsideloadingsummary():
    graph = Graph()

    cids = session.execute("SELECT DISTINCT content_id from content_sideloading_summary")
    for cid in cids:
        uid = cid['content_id']
        print("** Content:",uid)

        node=Node("Content",id=uid)
        graph.merge(node,"Content","id")

        contentDict = session.execute("SELECT * from content_sideloading_summary WHERE id='" + uid + "'")[0]
        cid = contentDict['content_id']
        
        #if (contentDict.has_key('total_count')):
        #    total_count = contentDict['total_count']
        #    node.properties['content_popularity'] = total_count
        #    node.push()
        node.properties['content_popularity'] = randint(1,100)
        node.push()
        
        print('content: ', cid, 'content_popularity: ',total_count)

# move concept map 
def moveConceptMap():
    # neo4j graph connector
    graph = Graph()
    # delete entire graph

    url="http://lp-sandbox.ekstep.org:8080/taxonomy-service/v2/analytics/domain/map"
    resp = requests.get(url).json()

    # move all concepts
    conceptList = resp["result"]["concepts"]
    for conceptDict in conceptList:
        identifier=None
    
        if(not conceptDict.has_key('identifier')):
            continue

        identifier = conceptDict['identifier']
        # create/find node
        node = graph.merge_one("Concept","id",identifier)

        if(conceptDict.has_key('subject')):
            subject = conceptDict['subject']
            node.properties["subject"]=subject
            node.push()

        if(conceptDict.has_key('objectType')):
            objectType = conceptDict['objectType']
            node.properties["objectType"]=objectType
            node.push()
       
        if(subject.lower()=='numeracy'):
            node.properties['tags'] = str(ASERdict_num[randint(1,5)])
        if(subject.lower()=='literacy'):
            node.properties['tags'] = str(ASERdict_lit[randint(1,5)])
        node.push()
        
        # move all relations
        relationList = resp["result"]["relations"]
    for relationDict in relationList:

        if (not relationDict.has_key('startNodeId') ):
            continue
        if (not relationDict.has_key('endNodeId') ):
            continue
        if (not relationDict.has_key('relationType') ):
            continue
        startNodeId = relationDict['startNodeId']
        endNodeId = relationDict['endNodeId']
        relationType = relationDict['relationType']
        node1 = graph.merge_one("Concept","id",startNodeId)
        node2 = graph.merge_one("Concept","id",endNodeId)
        graph.create(Relationship(node1, relationType, node2))

def moveContentModel():
    baseURL = "http://lp-sandbox.ekstep.org:8080/taxonomy-service/v2/analytics/getContent/"
    listURL = "http://lp-sandbox.ekstep.org:8080/taxonomy-service/v2/analytics/content/list"

    # neo4j graph connector
    graph = Graph()
    
    url = listURL
    resp = requests.get(url).json()
    # no of content
    contentList = resp["result"]["contents"]
    for contentListDict in contentList:
        # check if there is an identifier for this content
        if(not contentListDict.has_key('identifier')):
            continue
    
        # check if there is an identifier for this content
        identifier = contentListDict['identifier']

        # create a node for this Content
        node = graph.merge_one("Content","id",identifier)
       
        url = baseURL + identifier
        resp = requests.get(url)

        if(resp.status_code!=200):
            continue

        resp =  resp.json()

        concept=None
        Subject=None
        content_popularity=None
        
        contentDict = resp["result"]["content"]

    
        if(contentDict.has_key('concepts')):
            # this forms a "relationship" in the graph
            concepts = contentDict['concepts']
        
        if(contentDict.has_key('Subject')):
            # this forms a "relationship" in the graph
            Subject = contentDict['Subject']
            node.properties['Subject'] = Subject
            node.push()
            if(Subject.lower()=='numeracy'):
                node.properties['tags'] = str(ASERdict_num[randint(1,5)])
            if(Subject.lower()=='literacy'):
                node.properties['tags'] = str(ASERdict_lit[randint(1,5)])
            node.push()
            
        #node.properties['tags'] = "ASERlevel_"+str(randint(1,5))
        #node.push()
       
   
        #updating content popularity as a tag
        node.properties['content_popularity'] = randint(1,100)
        node.push()

print('................')
#moveConceptMap();
print('Concept Map populated')
# content model
print('................')
moveContentModel();
print('Content Model populated')
print('................')
#movecontentsideloadingsummary();
print('Content popularity score updated')

................
Concept Map populated
................
Content Model populated
................
Content popularity score updated


In [142]:
sub="numeracy"
level="ASERlevel_Beginner"
#content for exact match of level
query1="MATCH (b:Content) WHERE (lower(b.Subject)=~ '(?i)"+sub+"') AND ( has(b.tags)) AND ('"+level+"' IN (b.tags))  Return b.id AS Content, b.tags AS tag, b.content_popularity AS popularity ORDER BY popularity DESC"
query2="MATCH (b:Content) WHERE lower(b.Subject)=~ '(?i)"+sub+"' OPTIONAL MATCH (b) WHERE ( has(b.tags)) AND ('"+level+"' IN (b.tags))  Return b.id AS Content, b.tags AS tag, b.content_popularity AS popularity ORDER BY tag DESC,popularity DESC"
resp1= graph.cypher.execute(query1)
# content for match at domain level-optional match
resp2= graph.cypher.execute(query2)
resp2_array=np.asarray(resp2)
resp2_minusresp1=resp2_array[resp2_array[:,1]!=level,1]
#resp1=[]
if resp1:
    print('Recomended content for ASER level match:')
    resp1_content=np.asarray(resp1)[:,0]
    print(resp1_content,resp2_minusresp1)
else:
    print('Recomended content for \'subject\' level match :')
    print(resp2_minusresp1)


Recomended content for ASER level match:
(['numeracy_374', 'org.ekstep.numchart.worksheet',
       'org.ekstep.addobj.worksheet', 'org.ekstep.counting.worksheet',
       'org.ekstep.eng.num.activity', 'numeracy_365', 'numeracy_366',
       'numeracy_418', 'ASERlevel_Subtraction', 'ASERlevel_Subtraction',
       'ASERlevel_Multiplication', 'ASERlevel_Multiplication',
       'ASERlevel_Division', 'ASERlevel_Division', 'ASERlevel_Division',
       'ASERlevel_Division', 'ASERlevel_Division', 'ASERlevel_Addition',
       'ASERlevel_Addition'], dtype=object)
